In [ ]:
# To be able to make edits to repo without having to restart notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, mannwhitneyu, wilcoxon, ttest_rel, ttest_ind
import seaborn as sns
from matplotlib.patches import FancyArrowPatch
from matplotlib.colors import ColorConverter

remap_path = os.getcwd()
prototype_path = os.path.abspath(os.path.join(remap_path, os.pardir))
project_path = os.path.abspath(os.path.join(prototype_path, os.pardir))
lab_path = os.path.abspath(os.path.join(project_path, os.pardir))
sys.path.append(project_path)
os.chdir(project_path)
sys.path.append(project_path)
print(project_path)

In [ ]:
from _prototypes.cell_remapping.src.MEC_naming import MEC_naming_format, extract_name_mec

In [ ]:
# csv path
remapping_csv_path = r"C:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\full_32_permute.xlsx"

# load csv
remapping_df = pd.read_excel(remapping_csv_path)

# csv path
shuffle_spatial_path = r"C:\Users\aaoun\OneDrive - cumc.columbia.edu\Desktop\HussainiLab\neuroscikit_test_data\MEC_spatial_shuffles_combined_new.xlsx"

# load csv
shuffle_spatial_df = pd.read_excel(shuffle_spatial_path)

In [ ]:
# shuffle_spatial_df['p_value_information'] = 1 - shuffle_spatial_df['p_value_information']
# shuffle_spatial_df['p_value_sparsity'] = 1 - shuffle_spatial_df['p_value_sparsity']
# shuffle_spatial_df['p_value_selectivity'] = 1 - shuffle_spatial_df['p_value_selectivity']
# shuffle_spatial_df['p_value_coherence'] = 1 - shuffle_spatial_df['p_value_coherence']


In [ ]:
# Need to match neurofunc and remapping
import re
from scripts.batch_map.LEC_naming import LEC_naming_format, extract_name_lec

def _check_single_format(filename, fformat, fxn):
    if re.match(str(fformat), str(filename)) is not None:
        return list(fxn(filename))

# For neurofunc need to add extract, date, depth, name, stim 
# iterate thru rows of shuffle_spatial_df and extract from signature
for i, row in shuffle_spatial_df.iterrows():
    # extract
    fname = row['Session']
    name = extract_name_mec(fname)
    formats = MEC_naming_format
    for fformat in list(formats.keys()):
        checked = _check_single_format(fname, fformat, formats[fformat])
        if checked is not None:
            break
    stim, depth, name, date = checked
    
    shuffle_spatial_df.at[i, 'date'] = date
    shuffle_spatial_df.at[i, 'depth'] = depth
    shuffle_spatial_df.at[i, 'name'] = name
    shuffle_spatial_df.at[i, 'stim'] = stim


In [ ]:
import pandas as pd

# Define the common columns
neurofunc_row_identifiers = ['name', 'Tetrode', 'Cell ID']
remapping_row_identifiers = ['name', 'tetrode', 'unit_id']

for row in neurofunc_row_identifiers:
    shuffle_spatial_df[row] = shuffle_spatial_df[row].astype(str)
for row in remapping_row_identifiers:
    remapping_df[row] = remapping_df[row].astype(str)
shuffle_spatial_df['Session'] = shuffle_spatial_df['Session'].astype(str)
remapping_df['signature'] = remapping_df['signature'].astype(str)


In [ ]:
remapping_df.columns

In [ ]:

# Create an empty list to store the merged rows
merged_rows = []

# Iterate through each row in remapping_df
counter = 0
for remapping_row in remapping_df.itertuples(index=False):
    # Extract the identifiers from the remapping row
    remapping_identifiers = [getattr(remapping_row, col) for col in remapping_row_identifiers]

    # Find the matching row in shuffle_spatial_df based on the identifiers
    matching_row = shuffle_spatial_df.loc[
        (shuffle_spatial_df[neurofunc_row_identifiers] == remapping_identifiers).all(axis=1)
    ]

    # Check if a matching row was found
    if len(matching_row) == 0:
        print("Found {} matching rows for {}".format(len(matching_row), remapping_identifiers))
        print(matching_row['Session'], getattr(remapping_row, 'signature'))
        # stop()
    elif len(matching_row) > 1:
        # check all rows are matching on spatial info score 
        print(matching_row)
        prev = None
        for i, row in matching_row.iterrows():
            if prev is None:
                prev = row['spike_count']
            else:
                if prev != row['spike_count']:
                    print('multiple matches')
                    print(matching_row)
                    # stop()
        
    # Join the matching rows
    # merged_row = pd.concat([matching_row, pd.DataFrame([remapping_row], columns=remapping_df.columns)], axis=1)
    merged_rows.append(matching_row)
    counter += 1

    if len(merged_rows) != counter:
        print('error {} != {}'.format(len(merged_rows), counter))
        print(matching_row)


# Create the merged dataframe
merged_df = pd.concat(merged_rows, ignore_index=True)

# Print the merged dataframe
# print(merged_df)


In [ ]:
# shuffle_spatial_df['isSpatial'] = 0
# spatial_filtered = shuffle_spatial_df.loc[shuffle_spatial_df['p_value_information'] < 0.05]
# spatial_identifiers = spatial_filtered.groupby(neurofunc_row_identifiers).count().reset_index()[neurofunc_row_identifiers]
# for i, row in spatial_identifiers.iterrows():
#     matching_rows = remapping_df[remapping_row_identifiers].eq(row[neurofunc_row_identifiers]).all(axis=1)
#     remapping_df.loc[matching_rows, 'isSpatial'] = 1

# shuffle_spatial_df['isSparse'] = 0
# sparsity_filtered = shuffle_spatial_df.loc[shuffle_spatial_df['p_value_sparsity'] < 0.05]
# sparsity_identifiers = sparsity_filtered.groupby(neurofunc_row_identifiers).count().reset_index()[neurofunc_row_identifiers]
# for i, row in sparsity_identifiers.iterrows():
#     matching_rows = remapping_df[remapping_row_identifiers].eq(row[neurofunc_row_identifiers]).all(axis=1)
#     remapping_df.loc[matching_rows, 'isSparse'] = 1

# shuffle_spatial_df['isSelective'] = 0
# selectivity_filtered = shuffle_spatial_df.loc[shuffle_spatial_df['p_value_selectivity'] < 0.05]
# selectivity_identifiers = selectivity_filtered.groupby(neurofunc_row_identifiers).count().reset_index()[neurofunc_row_identifiers]
# for i, row in selectivity_identifiers.iterrows():
#     matching_rows = remapping_df[remapping_row_identifiers].eq(row[neurofunc_row_identifiers]).all(axis=1)
#     remapping_df.loc[matching_rows, 'isSelective'] = 1

# shuffle_spatial_df['isCoherent'] = 0
# coherence_filtered = shuffle_spatial_df.loc[shuffle_spatial_df['p_value_coherence'] < 0.05]
# coherence_identifiers = coherence_filtered.groupby(neurofunc_row_identifiers).count().reset_index()[neurofunc_row_identifiers]
# for i, row in coherence_identifiers.iterrows():
#     matching_rows = remapping_df[remapping_row_identifiers].eq(row[neurofunc_row_identifiers]).all(axis=1)
#     remapping_df.loc[matching_rows, 'isCoherent'] = 1


In [ ]:
ses_id_dict = {}
unq_names = shuffle_spatial_df['name'].unique()
unq_name_counts = {x:1 for x in unq_names}
for ses_id in np.sort(shuffle_spatial_df['Session'].unique()):
    nme = ses_id.split('_')[0]
    ses_id_dict[ses_id] = 'session_' + str(unq_name_counts[nme])
    unq_name_counts[nme] += 1

shuffle_spatial_df['session_id'] = ''
shuffle_spatial_df['group'] = ''

for i in range(len(shuffle_spatial_df)):
    shuffle_spatial_df.loc[i,'session_id'] = ses_id_dict[shuffle_spatial_df.iloc[i]['Session']]

app_ki = ['1-13', '1-14', '1a27', '1-30', '1a35', '1a37']
control = ['1-20', '1-24', '1-25', '1-28', '1-34', '1a23', '1a40']

for i in range(len(remapping_df)):
    nme = remapping_df.iloc[i]['name']
    if nme in control:
        remapping_df.loc[i,'group'] = 'control'
    else:
        assert nme in app_ki
        remapping_df.loc[i,'group'] = 'app_ki'
    ses1 = eval(remapping_df.iloc[i]['session_ids'])[0]
    ses2 = eval(remapping_df.iloc[i]['session_ids'])[1]

    ses1_match = shuffle_spatial_df.loc[(shuffle_spatial_df['session_id'] == ses1) & (shuffle_spatial_df['name'] == remapping_df.iloc[i]['name']) & (shuffle_spatial_df['Tetrode'] == remapping_df.iloc[i]['tetrode']) & (shuffle_spatial_df['Cell ID'] == remapping_df.iloc[i]['unit_id'])]
    ses2_match = shuffle_spatial_df.loc[(shuffle_spatial_df['session_id'] == ses2) & (shuffle_spatial_df['name'] == remapping_df.iloc[i]['name']) & (shuffle_spatial_df['Tetrode'] == remapping_df.iloc[i]['tetrode']) & (shuffle_spatial_df['Cell ID'] == remapping_df.iloc[i]['unit_id'])]

    assert len(ses1_match) == 1, 'ses1_match: {}'.format(len(ses1_match))
    assert len(ses2_match) == 1, 'ses2_match: {}'.format(len(ses2_match))

    for ky in ['information', 'sparsity', 'selectivity', 'coherence']:
        score_origin_session = ses1_match.iloc[0][ky]
        score_target_session = ses2_match.iloc[0][ky]
        remapping_df.loc[i, ky + '_origin_session'] = score_origin_session
        remapping_df.loc[i, ky + '_target_session'] = score_target_session

        shuffled_score_mean_origin_session = ses1_match.iloc[0]['shuffled_' + ky + '_mean']
        shuffled_score_mean_target_session = ses2_match.iloc[0]['shuffled_' + ky + '_mean']
        remapping_df.loc[i, ky + '_reference_mean_origin_session'] = shuffled_score_mean_origin_session
        remapping_df.loc[i, ky + '_reference_mean_target_session'] = shuffled_score_mean_target_session

        shuffled_score_sd_origin_session = ses1_match.iloc[0]['shuffled_' + ky + '_std']
        shuffled_score_sd_target_session = ses2_match.iloc[0]['shuffled_' + ky + '_std']
        remapping_df.loc[i, ky + '_reference_sd_origin_session'] = shuffled_score_sd_origin_session
        remapping_df.loc[i, ky + '_reference_sd_target_session'] = shuffled_score_sd_target_session

        shuffled_score_p_value_origin_session = ses1_match.iloc[0]['p_value_' + ky]
        shuffled_score_p_value_target_session = ses2_match.iloc[0]['p_value_' + ky]
        remapping_df.loc[i, ky + '_quantile_origin_session'] = shuffled_score_p_value_origin_session
        remapping_df.loc[i, ky + '_quantile_target_session'] = shuffled_score_p_value_target_session

    

In [ ]:
remapping_df.keys()

In [ ]:
# save as csv
remapping_df.to_excel('remapping_shuffle_merged.xlsx', index=False)

In [ ]:
remapping_df['group']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi

# Generate random centroids (x, y positions between 0 and 1)
np.random.seed(0)
num_centroids = 20
centroids = np.random.rand(num_centroids, 2)

# Create a grid for visualization
x_grid, y_grid = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 1, 100))
grid_points = np.column_stack((x_grid.flatten(), y_grid.flatten()))

# Calculate Voronoi diagram
vor = Voronoi(centroids)

# Get the Voronoi regions and plot them
plt.figure(figsize=(8, 8))
for region in vor.regions:
    if not -1 in region and len(region) > 0:
        polygon = [vor.vertices[i] for i in region]
        plt.fill(*zip(*polygon), edgecolor='black', linewidth=1, alpha=0.5)

# Plot the centroids
plt.scatter(centroids[:, 0], centroids[:, 1], color='red', marker='o', s=50, label='Initial Centroids')

# Plot settings
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.gca().set_aspect('equal', adjustable='box')
plt.title('Centroids Converging to Hexagonal Mesh')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.grid()

# Show the plot
plt.show()

In [ ]:
remapping_df['isSpatial'] = 0

# any session pvalue < 0.05 then cell is spatial
# spatial_filtered = shuffle_spatial_df.loc[shuffle_spatial_df['p_value_information'] < 0.05]
# spatial_identifiers = spatial_filtered.groupby(neurofunc_row_identifiers).count().reset_index()[neurofunc_row_identifiers]

# only first session pvalue < 0.05 then cell is spatial
spatial_identifiers = shuffle_spatial_df.groupby(neurofunc_row_identifiers).agg({'p_value_information': 'first'}).reset_index()
spatial_identifiers = spatial_identifiers.loc[spatial_identifiers['p_value_information'] < 0.05]
for i, row in spatial_identifiers.iterrows():
    si_done = False
    for j, val in remapping_df[remapping_row_identifiers].iterrows():
        if list(row[neurofunc_row_identifiers].to_numpy()) == list(val.to_numpy()):
            remapping_df.loc[j, 'isSpatial'] = 1
            si_done = True
    if si_done == False:
        print('error spatial')

remapping_df['isSparse'] = 0
# sparsity_filtered = shuffle_spatial_df.loc[shuffle_spatial_df['p_value_sparsity'] < 0.05]
# sparsity_identifiers = sparsity_filtered.groupby(neurofunc_row_identifiers).count().reset_index()[neurofunc_row_identifiers]
sparsity_identifiers = shuffle_spatial_df.groupby(neurofunc_row_identifiers).agg({'p_value_sparsity': 'first'}).reset_index()
sparsity_identifiers = sparsity_identifiers.loc[sparsity_identifiers['p_value_sparsity'] < 0.05]
for i, row in sparsity_identifiers.iterrows():
    si_done = False
    for j, val in remapping_df[remapping_row_identifiers].iterrows():
        if list(row[neurofunc_row_identifiers].to_numpy()) == list(val.to_numpy()):
            remapping_df.loc[j, 'isSparse'] = 1
            si_done = True
    if si_done == False:
        print('error sparsity')

remapping_df['isSelective'] = 0
# selectivity_filtered = shuffle_spatial_df.loc[shuffle_spatial_df['p_value_selectivity'] < 0.05]
# selectivity_identifiers = selectivity_filtered.groupby(neurofunc_row_identifiers).count().reset_index()[neurofunc_row_identifiers]
selectivity_identifiers = shuffle_spatial_df.groupby(neurofunc_row_identifiers).agg({'p_value_selectivity': 'first'}).reset_index()
selectivity_identifiers = selectivity_identifiers.loc[selectivity_identifiers['p_value_selectivity'] < 0.05]
for i, row in selectivity_identifiers.iterrows():
    si_done = False
    for j, val in remapping_df[remapping_row_identifiers].iterrows():
        if list(row[neurofunc_row_identifiers].to_numpy()) == list(val.to_numpy()):
            remapping_df.loc[j, 'isSelective'] = 1
            si_done = True
    if si_done == False:
        print('error selectivity')

remapping_df['isCoherent'] = 0
# coherence_filtered = shuffle_spatial_df.loc[shuffle_spatial_df['p_value_coherence'] < 0.05]
# coherence_identifiers = coherence_filtered.groupby(neurofunc_row_identifiers).count().reset_index()[neurofunc_row_identifiers]
coherence_identifiers = shuffle_spatial_df.groupby(neurofunc_row_identifiers).agg({'p_value_coherence': 'first'}).reset_index()
coherence_identifiers = coherence_identifiers.loc[coherence_identifiers['p_value_coherence'] < 0.05]
for i, row in coherence_identifiers.iterrows():
    si_done = False
    for j, val in remapping_df[remapping_row_identifiers].iterrows():
        if list(row[neurofunc_row_identifiers].to_numpy()) == list(val.to_numpy()):
            remapping_df.loc[j, 'isCoherent'] = 1
            si_done = True
    if si_done == False:
        print('error coherence')


In [ ]:
# save 
remapping_df.to_excel('filtered_first_ses_greater.xlsx', index=False)

In [ ]:
shuffle_spatial_df.columns

In [ ]:
fig = plt.figure(figsize=(8, 6))

ax = fig.add_subplot(2, 2, 1)
cts = ax.hist(np.log(shuffle_spatial_df['shuffled_information_mean']), bins=100)
ax.vlines(np.log(np.mean(shuffle_spatial_df['shuffled_information_mean'])), 0, np.max(cts[0]),color='r')
ax.set_xlabel('Shuffled Information Means')
ax.set_ylabel('Count')

ax = fig.add_subplot(2, 2, 2)
ax.hist(np.log(shuffle_spatial_df['shuffled_sparsity_mean']), bins=100)
ax.set_xlabel('Shuffled Sparsity Means')
ax.set_ylabel('Count')

ax = fig.add_subplot(2, 2, 3)
ax.hist(np.log(shuffle_spatial_df['shuffled_selectivity_mean']), bins=100)
ax.set_xlabel('Shuffled Selectivity Means')
ax.set_ylabel('Count')

ax = fig.add_subplot(2, 2, 4)
ax.hist(np.log(shuffle_spatial_df['shuffled_coherence_mean']), bins=100)
ax.set_xlabel('Shuffled Coherence Means')
ax.set_ylabel('Count')

fig.tight_layout()
plt.show()

In [ ]:
print(remapping_df['isSpatial'].value_counts())
print(remapping_df['isSparse'].value_counts())
print(remapping_df['isSelective'].value_counts())
print(remapping_df['isCoherent'].value_counts())

In [ ]:
print(remapping_df['isSpatial'].value_counts())
print(remapping_df['isSparse'].value_counts())
print(remapping_df['isSelective'].value_counts())
print(remapping_df['isCoherent'].value_counts())

In [ ]:
# spatial_df = remapping_df.groupby(remapping_row_identifiers)
# filtered_spatial_df = pd.DataFrame(columns=remapping_df.columns)

# for i, grp in spatial_df:
#     ses1_p_info = grp['p_value_information'].iloc[0]
#     if ses1_p_info < 0.05:
#         filtered_spatial_df = filtered_spatial_df.append(grp)